In [63]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
    }
    .atcenter{
        text-align:center !important;
    }
    .imp{
        color:red;
        font-weight:bolder;
    }
</style>

# 天猫订单数据分析

本数据集共手机了发生在一个月内的28010条数据,包含  
- 订单编号 
- 总金额  
- 买家实际支付金额: 总金额-退款金额  
- 收货地址:各个省份
- 订单创建时间
- 订单付款时间
- 退款金额: 付款后申请退款的金额,如未付款,退款金额为0

## 导入需要的包

In [64]:
from pyecharts.globals import CurrentConfig,NotebookType
import pyecharts.options as opts
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts.charts import Bar,Line,Pie,Map,Timeline
import pandas as pd
import numpy as np

## 数据准备阶段

### 先读入数据

In [65]:
data = pd.read_csv('../others/tmall_order_report.csv')

In [66]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2022-03-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2022-03-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8


In [67]:
data.tail()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
28005,28006,37.0,37.0,四川省,2022-03-27 00:01:00,2022-03-27 00:01:10,0.0
28006,28007,69.0,0.0,上海,2022-03-27 00:00:18,NaN,0.0
28007,28008,69.0,0.0,上海,2022-03-27 00:00:17,NaN,0.0
28008,28009,37.0,37.0,辽宁省,2022-03-27 00:00:09,2022-03-27 00:00:17,0.0
28009,28010,37.0,37.0,广东省,2022-03-27 00:00:06,2022-03-27 00:00:11,0.0


In [68]:
data.shape

(28010, 7)

In [69]:
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额'], dtype='object')

### 查看数据列中是否有空值

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


#### ↑可以看出是有空值的,如索引5订单付款时间列非空的数据只有24087个,不足28010个

### 数据处理,去掉列明中的文字左右空格

In [71]:
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额'], dtype='object')

#### 可以看到收货地址和订单付款时间右边都有空格

In [72]:
data.columns.str.strip() #利用.strip()去除列名的左右空格

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

#### 去除成功

### 查看是否有重复值,使用duplicated()方法,如果不重复就是False

In [73]:
data.duplicated() #这样看会有很多省略号

0        False
1        False
2        False
3        False
4        False
         ...  
28005    False
28006    False
28007    False
28008    False
28009    False
Length: 28010, dtype: bool

In [74]:
data.duplicated().sum() 
#将非空的行进行求和,如果说最后的数值是0则说明全都不重复

np.int64(0)

### 查看是否有缺失值

In [75]:
np.any(data.isnull()) 
#这个的含义是如果数据(data)有任何(any)空(isnull())就返回True

np.True_

#### 查看每一列的空值数量

In [76]:
data.isnull().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

## 数据可视化

In [77]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2022-03-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2022-03-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8


### 用地图展示各省份的订单量

#### 首先将原来的带空格的列名称换掉,刚才没进行这一步操作

In [78]:
data.columns = data.columns.str.strip()

In [79]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2022-03-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2022-03-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8


#### 可以通过省份出现的次数得出每个省份的订单量是多少

In [80]:
data['收货地址'].value_counts()

收货地址
上海          3353
广东省         2463
江苏省         2126
浙江省         2061
北京          2054
四川省         2019
山东省         1804
辽宁省         1187
天津          1153
湖南省         1099
河北省         1083
重庆          1036
河南省          966
云南省          778
安徽省          609
陕西省          536
福建省          489
山西省          465
广西壮族自治区      436
江西省          411
吉林省          401
黑龙江省         379
贵州省          345
内蒙古自治区       215
海南省          178
甘肃省          167
湖北省           75
新疆维吾尔自治区      58
宁夏回族自治区       42
青海省           19
西藏自治区          3
Name: count, dtype: int64

In [81]:
def province_map(p):
    if p in ['北京','上海','天津','重庆']:
        return p+'市'
    else:
        return p
data['收货地址']=data['收货地址'].map(province_map)  #注意,也是把原来的改掉
#对数据中的'收货地址'这一列的数据进行整体操作,操作规则为指定的函数province_map
#回顾一下map(function,iterable) 对可迭代数据进行整体操作,指定函数也是可以的

In [82]:
# 再次查看数据
data['收货地址'].value_counts()

收货地址
上海市         3353
广东省         2463
江苏省         2126
浙江省         2061
北京市         2054
四川省         2019
山东省         1804
辽宁省         1187
天津市         1153
湖南省         1099
河北省         1083
重庆市         1036
河南省          966
云南省          778
安徽省          609
陕西省          536
福建省          489
山西省          465
广西壮族自治区      436
江西省          411
吉林省          401
黑龙江省         379
贵州省          345
内蒙古自治区       215
海南省          178
甘肃省          167
湖北省           75
新疆维吾尔自治区      58
宁夏回族自治区       42
青海省           19
西藏自治区          3
Name: count, dtype: int64

#### 统计各个省份的订单量

In [83]:
data['订单付款时间'].notnull() 
#首先拿到订单付款时间不为空的数据

0        False
1         True
2        False
3         True
4         True
         ...  
28005     True
28006    False
28007    False
28008     True
28009     True
Name: 订单付款时间, Length: 28010, dtype: bool

In [84]:
data[data['订单付款时间'].notnull()] #利用掩码操作,只保留非空的数据

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8
5,6,327.7,148.9,浙江省,2022-03-20 23:56:39,2022-03-20 23:56:53,178.8
6,7,357.0,357.0,天津市,2022-03-20 23:56:36,2022-03-20 23:56:40,0.0
...,...,...,...,...,...,...,...
28002,28003,77.0,77.0,重庆市,2022-03-27 00:02:39,2022-03-27 00:03:27,0.0
28003,28004,157.0,157.0,山东省,2022-03-27 00:01:42,2022-03-27 00:01:47,0.0
28005,28006,37.0,37.0,四川省,2022-03-27 00:01:00,2022-03-27 00:01:10,0.0
28008,28009,37.0,37.0,辽宁省,2022-03-27 00:00:09,2022-03-27 00:00:17,0.0


In [85]:
result = data[data['订单付款时间'].notnull()].groupby('收货地址')[['订单编号']].count()
#保留非空的数据,并进行分组,按照收货地址来分组

In [86]:
result

,订单编号
收货地址,
上海市,3060
云南省,667
内蒙古自治区,176
北京市,1853
吉林省,336
四川省,1752
天津市,1031
宁夏回族自治区,40
安徽省,528


In [87]:
result.to_dict()

{'订单编号': {'上海市': 3060,
  '云南省': 667,
  '内蒙古自治区': 176,
  '北京市': 1853,
  '吉林省': 336,
  '四川省': 1752,
  '天津市': 1031,
  '宁夏回族自治区': 40,
  '安徽省': 528,
  '山东省': 1484,
  '山西省': 395,
  '广东省': 2022,
  '广西壮族自治区': 353,
  '新疆维吾尔自治区': 43,
  '江苏省': 1845,
  '江西省': 331,
  '河北省': 885,
  '河南省': 792,
  '浙江省': 1822,
  '海南省': 156,
  '湖北省': 57,
  '湖南省': 935,
  '甘肃省': 132,
  '福建省': 425,
  '西藏自治区': 2,
  '贵州省': 286,
  '辽宁省': 1012,
  '重庆市': 896,
  '陕西省': 441,
  '青海省': 18,
  '黑龙江省': 312}}

#### 将其转换为字典结构,只保留''

In [88]:
result2 = result.to_dict()['订单编号']
result2

{'上海市': 3060,
 '云南省': 667,
 '内蒙古自治区': 176,
 '北京市': 1853,
 '吉林省': 336,
 '四川省': 1752,
 '天津市': 1031,
 '宁夏回族自治区': 40,
 '安徽省': 528,
 '山东省': 1484,
 '山西省': 395,
 '广东省': 2022,
 '广西壮族自治区': 353,
 '新疆维吾尔自治区': 43,
 '江苏省': 1845,
 '江西省': 331,
 '河北省': 885,
 '河南省': 792,
 '浙江省': 1822,
 '海南省': 156,
 '湖北省': 57,
 '湖南省': 935,
 '甘肃省': 132,
 '福建省': 425,
 '西藏自治区': 2,
 '贵州省': 286,
 '辽宁省': 1012,
 '重庆市': 896,
 '陕西省': 441,
 '青海省': 18,
 '黑龙江省': 312}

#### 还要将其转换为二维数组的形式

In [89]:
[*result2.items()]

[('上海市', 3060),
 ('云南省', 667),
 ('内蒙古自治区', 176),
 ('北京市', 1853),
 ('吉林省', 336),
 ('四川省', 1752),
 ('天津市', 1031),
 ('宁夏回族自治区', 40),
 ('安徽省', 528),
 ('山东省', 1484),
 ('山西省', 395),
 ('广东省', 2022),
 ('广西壮族自治区', 353),
 ('新疆维吾尔自治区', 43),
 ('江苏省', 1845),
 ('江西省', 331),
 ('河北省', 885),
 ('河南省', 792),
 ('浙江省', 1822),
 ('海南省', 156),
 ('湖北省', 57),
 ('湖南省', 935),
 ('甘肃省', 132),
 ('福建省', 425),
 ('西藏自治区', 2),
 ('贵州省', 286),
 ('辽宁省', 1012),
 ('重庆市', 896),
 ('陕西省', 441),
 ('青海省', 18),
 ('黑龙江省', 312)]

In [90]:
#或者这样写也是可以的
list(result2.items())

[('上海市', 3060),
 ('云南省', 667),
 ('内蒙古自治区', 176),
 ('北京市', 1853),
 ('吉林省', 336),
 ('四川省', 1752),
 ('天津市', 1031),
 ('宁夏回族自治区', 40),
 ('安徽省', 528),
 ('山东省', 1484),
 ('山西省', 395),
 ('广东省', 2022),
 ('广西壮族自治区', 353),
 ('新疆维吾尔自治区', 43),
 ('江苏省', 1845),
 ('江西省', 331),
 ('河北省', 885),
 ('河南省', 792),
 ('浙江省', 1822),
 ('海南省', 156),
 ('湖北省', 57),
 ('湖南省', 935),
 ('甘肃省', 132),
 ('福建省', 425),
 ('西藏自治区', 2),
 ('贵州省', 286),
 ('辽宁省', 1012),
 ('重庆市', 896),
 ('陕西省', 441),
 ('青海省', 18),
 ('黑龙江省', 312)]

In [91]:
c = (
        Map()
        .add(
            '各省份订单量',
            [*result2.items()],
            'china',
            is_map_symbol_show=False, #隐藏各个省份的符号
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title='天猫订单各省份数量信息'
            ),
            # visualmap_opts=opts.VisualMapOpts(
            #     range_text=['多','少']
            # ),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(
                is_show = True
            )
        )
)

In [92]:
c.load_javascript()

In [93]:
c.render_notebook()

In [94]:
c.render('../others/tm.html')

'D:\\人工智能学习\\01机器学习\\pyecharts学习\\others\\tm.html'